In [1]:
cd ..

C:\Users\wpgur\OneDrive\프로그래밍\Back-End\tdatlib


In [2]:
# For Library
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
from datetime import timedelta

# For Testing
import tdatlib as tdat
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important;}</style>"))

# LIB DEV

## Functionality

In [3]:
def align(l:pd.Series or pd.DataFrame, r:pd.Series or pd.DataFrame):
    join = pd.concat(objs={'L':l, 'R':r}, axis=1).dropna()
    join = join[join.index.isin(l.index)]
    return join.L, join.R

In [4]:
def normalize(time_series:pd.Series or np.array, a:float=0, b:float=1):
    return (b - a) * (time_series - time_series.min()) / (time_series.max() - time_series.min()) + a

In [5]:
def price_rel(time_series:pd.Series or pd.DataFrame, init:float=100):
    return init * ((time_series.dropna().pct_change() + 1).cumprod() - 1)

In [15]:
def ohlcv2hlc(ohlcv:pd.DataFrame) -> pd.Series:
    if not ('종가' in ohlcv.columns and '고가' in ohlcv.columns and '저가' in ohlcv.columns):
        raise KeyError
    return (1/3) * ohlcv.종가 + (1/3) * ohlcv.고가 + (1/3) * ohlcv.저가

In [7]:
def corrcoeff(l:pd.Series, r:pd.Series) -> float:
    return pd.concat(objs=align(l, r), axis=1).corr(method='pearson', min_periods=1).iloc[0, 1]

In [8]:
def weighted_corrcoeff(joined:pd.DataFrame) -> float:
    days = (joined.index[-1] - joined.index[0]).days
    gaps = [0, 92, 183, 365, 365 * 2, 365 * 3, 365 * 5, 365 * 10]
    n = [i + 1 for i in range(len(gaps)-1) if gaps[0] < days <= gaps[i + 1]][0]

    coeffs = list()
    for g in gaps[1:n]:
        coeff = joined[joined.index >= (joined.index[-1] - timedelta(g))].corr().iloc[0, 1]
        coeffs.append(coeff)
    coeffs.append(joined.corr().iloc[0, 1])
    return np.array(coeffs).mean()

In [47]:
def rolling_corrcoeff(l:pd.Series, r:pd.Series, month:int):
    prev_day = l.index[-1] - timedelta(days=int(month * 30.5))
    samples = len(r.index[r.index >= prev_day])
    index = np.arange(start=-samples, stop=samples + 1, step=1)

    dates = [r.index[-1] + timedelta(int(i)) for i in index]
    data = [[corrcoeff(l, r.shift(i)), i] for i in index]
    return pd.DataFrame(data=data, index=dates, columns=['corrcoef', 'steps'])


In [84]:
class corrmodel(object):
    
    def __init__(self, l:pd.Series, r:pd.Series):
        self.__l, self.__r = align(l, r)
        self.__joined = pd.concat(objs=[self.__l, self.__r], axis=1)
        return
    
    @property
    def coeff(self) -> float:
        if not hasattr(self, '__coeff'):
            self.__setattr__('__coeff', self.__joined.corr().iloc[0, 1])
        return self.__getattribute__('__coeff')
    
    @property
    def coeffmd(self) -> float:
        if not hasattr(self, '__coeffmd'):
            if len(self.__joined) < 93:
                return self.coeff
            self.__setattr__('__coeffmd', weighted_corrcoeff(self.__joined))
        return self.__getattribute__('__coeffmd')
    
    @property
    def coeffrl(self) -> float:
        coeff, _, __ = self._rolling_coeff()
        return coeff
    
    def _rolling_coeff(self) -> tuple:
        if not hasattr(self, '__coeffrl'):
            rc = rolling_corrcoeff(self.__l, self.__r, 6)
            rc['abs'] = rc.corrcoef.abs()
            row = rc[rc['abs'] == rc['abs'].max()]
            coeff, step = row.iloc[0, 0], row.iloc[0, 1]
            pivot = self.__r.index[-1 if step < 0 else 0]
            far = self.__r.index[step - 1 if step < 0 else step]
            self.__setattr__('__coeffrl', (coeff, pivot - far, step))
        return self.__getattribute__('__coeffrl')    
    
    def _trace_l(self) -> go.Scatter:
        return go.Scatter(
            x=self.__l.index, y=self.__l, name='left',
            visible=True, showlegend=True,
            xhoverformat='%Y/%m/%d', hovertemplate='%{x}<br>%{y}'
        )
    
    def _trace_r(self) -> go.Scatter:
        return go.Scatter(
            x=self.__r.index, y=self.__r, name='right',
            visible=True, showlegend=True,
            xhoverformat='%Y/%m/%d', hovertemplate='%{x}<br>%{y}'
        )
    
    def _trace_rshift(self) -> go.Scatter:
        _, _, step = self.coeffrl
        return go.Scatter(
            x=self.__r.shift(step).index, y=self.__r.shift(step), name='right<br>shifted',
            visible='legendonly', showlegend=True,
            xhoverformat='%Y/%m/%d', hovertemplate='%{x}<br>%{y}'
        )
    
    def _trace_scatter(self) -> go.Scatter:
        return go.Scatter(
            x=self.__l, y=self.__r, name='Scatter',
            meta=self.__joined.index, mode='markers',
            hovertemplate='%{meta}<br>x = %{x}<br>y = %{y}'
        )
    
    def show_aligned(self):
        fig = make_subplots(rows=1, cols=1, shared_xaxes=True, specs=[[{"type": "xy", "secondary_y": True}]])
        fig.add_trace(self._trace_l(), secondary_y=False)
        fig.add_trace(self._trace_r(), secondary_y=True)
        fig.add_trace(self._trace_rshift(), secondary_y=True)
        return fig
    
    def show_scatter(self):
        fig = go.Figure()
        fig.add_trace(self._trace_scatter())
        return fig

# TESTER

## BASIC DATASET

### Handler

In [10]:
ecos = tdat.macro.ecos()
fred = tdat.macro.fred()
index = tdat.market.index()

period = 5
ecos.period = fred.period = index.period = period

ticker = '105560'
stock = tdat.stock.kr(ticker=ticker)
stock.period = period

### Indicator

In [41]:
cols = ['시가', '고가', '저가', '종가']
kospi = index.kospi
krbank = index.bank
exchange = ecos.원달러환율
margin = ecos.load('121Y015', '총대출(당좌대출 제외)') - ecos.load('121Y013', '저축성수신(금융채 제외)')

ohlcv = stock.ohlcv
# pd.concat(objs={stock.name:ohlcv[cols], '코스피':kospi[cols], '환율':exchange, '은행':krbank[cols]}, axis=1)

## TESTING DATASET

In [86]:
left = krbank.copy()
# left = ohlcv.copy()

# right = exchange.copy()
right = margin.copy()

# mycorr = corrmodel(ohlcv2hlc(left), ohlcv2hlc(right))
mycorr = corrmodel(ohlcv2hlc(left), right)
# mycorr = corrmodel(right, ohlcv2hlc(left))
print(mycorr.coeff)
print(mycorr.coeffmd)
print(mycorr.coeffrl)

mycorr.show_aligned()
# mycorr.show_scatter()

0.6459166523387768
0.6459166523387768
(0.8111834293963843, Timedelta('-92 days +00:00:00'), 2)


## VISUALIZE

### Scatter

In [23]:
_x, _y = left.종가, right

x, y = normalize(_x, a=-1, b=1), normalize(_y, a=-1, b=1)
# x, y = _x.resample('M').last(), _y.resample('M').last()
# x = _x.resample('M').last().pct_change()
# y = _y.resample('M').last().pct_change()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x, y=y, name='산포도',
    mode='markers',
    
))
fig.update_layout(height=700)
fig.show()

### x-y

In [ ]:
y1 = price_rel(comparatee.종가)
y2 = price_rel(comparator.종가)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=y1.index, y=y1, name='LEFT',
    xhoverformat='%Y/%m/%d'
))

fig.add_trace(go.Scatter(
    x=y2.index, y=y2, name='RIGHT',
    xhoverformat='%Y/%m/%d'
))
fig.update_layout(height=700)
fig.show()

### x-y1, y2

In [24]:
y1 = left.종가
# y2 = right.종가
y2 = right

fig = make_subplots(
    rows=2, cols=1, row_width=[0.3, 0.7], vertical_spacing=0.02, shared_xaxes=False,
    specs=[
        [{"type": "xy", "secondary_y": True}],
        [{"type": "xy"}]
    ]
)

fig.add_trace(go.Scatter(
    x=y1.index, y=y1, name='LEFT',
    xhoverformat='%Y/%m/%d'
), row=1, col=1, secondary_y=False)

fig.add_trace(go.Scatter(
    x=y2.index, y=y2, name='RIGHT',
    xhoverformat='%Y/%m/%d'
), row=1, col=1, secondary_y=True)

fig.add_trace(go.Scatter(
    x=rl.days, y=rl.corrcoef, name='Rolling'
), row=2, col=1)

fig.update_layout(height=700)
fig.show()